In [23]:
import numpy
import transformers

In [24]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [25]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [26]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Using", torch.cuda.device_count(), "GPUs!")

model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

Using device: cuda:1
Using 4 GPUs!


In [27]:
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [28]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
from torch.optim import AdamW

class GPT2QADataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.inputs = []
        self.attentions = []

        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # print(line)
                encodings = tokenizer.encode_plus(line, add_special_tokens=True, max_length=max_length, return_tensors='pt', truncation=True, padding='max_length')
                self.inputs.append(encodings['input_ids'].squeeze(0))
                self.attentions.append(encodings['attention_mask'].squeeze(0))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.attentions[idx]

In [29]:
file_path = '../data/gpt2_finetune_data-4.txt'
max_length = 512
dataset = GPT2QADataset(file_path, tokenizer, max_length)
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

optimizer = AdamW(model.parameters(), lr=0.0001)

epochs = 1
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [30]:
dataset.__getitem__(0)

(tensor([24361,    25,  5338,   318,   379,  2526,   329,   406, 20896, 13733,
         13370,   609, 10145,   296,  3101, 11815,   357,  5639,    44, 19427,
          5633, 23998,    25, 22228,    44,    53, 16079,   460,  3051,   706,
          7111,   284,  4713, 18922,    11,  3102, 37840,    11, 40333,    11,
           393, 46282,  5696,   422, 14112, 41093,    13, 41653,   743,   635,
          3051,   618,   777,  5696,   389,  3264,  5495,   656,  5445,  4168,
            11,   262,  9686,    11,   262,  2951,    11,   393,   262,  5422,
            11,   393, 14572,    11,  2884,   262, 13197,   286,   281, 14112,
         50074,    13,  7755,    12,  1462,    12,  6259, 11478,   468,   407,
           587,  2098,    11,   351,   262,  6631,   286, 11723, 11478,   422,
         14112,  2802,   284, 31076,    11,   290,  8365,    11,   832,  1618,
         23319,   341,    13,   198, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 5

In [31]:
first_batch = next(iter(dataloader))
print(first_batch)

[tensor([[24361,    25,  1867,  ..., 50256, 50256, 50256],
        [24361,    25,   314,  ..., 50256, 50256, 50256],
        [24361,    25,   314,  ..., 50256, 50256, 50256],
        [24361,    25, 18435,  ..., 50256, 50256, 50256]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])]


In [32]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Check GPU

In [46]:
!nvidia-smi

Wed Dec  6 12:36:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     On  | 00000000:4F:00.0 Off |                    0 |
|  0%   39C    P0              74W / 300W |   1774MiB / 46068MiB |      0%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [34]:
!kill -9 1625571

/bin/bash: line 0: kill: (1625571) - No such process


In [35]:
# training_args = TrainingArguments(
#     output_dir='../model/checkpoints-gpt/results/',
#     overwrite_output_dir=True,
#     num_train_epochs=1,
#     per_device_train_batch_size=4,
#     save_steps=5000,
#     save_total_limit=2,
#     prediction_loss_only=True,
#     no_cuda=False
# )

# from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir='../model/checkpoints-gpt/results/',
#     overwrite_output_dir=True,
#     num_train_epochs=1,
#     per_device_train_batch_size=4,
#     save_steps=5000,
#     save_total_limit=2,
#     prediction_loss_only=True,
#     no_cuda=False,
#     logging_dir='../logs',
#     logging_steps=100,
# )


In [36]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=dataset,
# )

In [37]:
import os
model_save_path = '/projectnb/cs505ws/students/praneshj/final-project/model'
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

for epoch in range(epochs):
    epoch_loss = 0
    for i, batch in enumerate(tqdm(dataloader)):
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        outputs = model(input_ids, labels=input_ids, attention_mask=attention_mask)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        if (i + 1) % 1000 == 0:
            print(f"Iteration {i + 1}: Loss = {loss.item()}")

    print(f"Epoch {epoch + 1} completed. Avg loss: {epoch_loss / len(dataloader)}")

    model_save_epoch_path = os.path.join(model_save_path, f"gpt_medical_checkpoint_{epoch+1}")
    os.makedirs(model_save_epoch_path, exist_ok=True)
    model.save_pretrained(model_save_epoch_path)
    tokenizer.save_pretrained(model_save_epoch_path)


  2%|▏         | 1001/45402 [03:01<2:14:52,  5.49it/s]

Iteration 1000: Loss = 1.5201516151428223


  4%|▍         | 2001/45402 [06:03<2:11:41,  5.49it/s]

Iteration 2000: Loss = 1.2571905851364136


  7%|▋         | 3001/45402 [09:06<2:08:56,  5.48it/s]

...

Iteration 44000: Loss = 1.5793628692626953


 99%|█████████▉| 45001/45402 [2:16:35<01:13,  5.49it/s]

Iteration 45000: Loss = 1.2125040292739868


100%|██████████| 45402/45402 [2:17:48<00:00,  5.49it/s]


Epoch 1 completed. Avg loss: 1.0295281304699393


In [38]:
# model_save_path = '../model/gpt2_medical'
# model.save_pretrained(model_save_path)
# tokenizer.save_pretrained(model_save_path)

## Epoch-2

In [48]:
new_lr = 0.00005
for param_group in optimizer.param_groups:
    param_group['lr'] = new_lr

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5e-05
    maximize: False
    weight_decay: 0.01
)

In [57]:
import os
model_save_path = '/projectnb/cs505ws/students/praneshj/final-project/model'
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model=model.to(device)
for epoch in range(2, 3):
    epoch_loss = 0
    for i, batch in enumerate(tqdm(dataloader)):
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        outputs = model(input_ids, labels=input_ids, attention_mask=attention_mask)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        if (i + 1) % 1000 == 0:
            print(f"Iteration {i + 1}: Loss = {loss.item()}")

    print(f"Epoch {epoch + 1} completed. Avg loss: {epoch_loss / len(dataloader)}")

    model_save_epoch_path = os.path.join(model_save_path, f"gpt_medical_checkpoint_{epoch+1}")
    os.makedirs(model_save_epoch_path, exist_ok=True)
    model.save_pretrained(model_save_epoch_path)
    tokenizer.save_pretrained(model_save_epoch_path)


  2%|▏         | 1001/45402 [02:39<1:58:57,  6.22it/s]

Iteration 1000: Loss = 0.1680896133184433


  4%|▍         | 2001/45402 [05:20<1:55:51,  6.24it/s]

Iteration 2000: Loss = 0.834034264087677


  7%|▋         | 3001/45402 [08:01<1:53:13,  6.24it/s]

...

Iteration 44000: Loss = 0.9359433054924011


 99%|█████████▉| 45001/45402 [2:00:13<01:04,  6.21it/s]

Iteration 45000: Loss = 0.6298321485519409


100%|██████████| 45402/45402 [2:01:18<00:00,  6.24it/s]


Epoch 3 completed. Avg loss: 0.9066647641256719


## Inference

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_save_path="/Users/praneshjayasundar/Documents/Gunner/Boston-University/Fall-2023/Student/CS505/final-project/health-assistant/model/gpt_medical_checkpoint_1"
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_save_path)

In [3]:
question = "I have severe cold, which medicine should I take?"
input_text = f"Question: {question} Answer:"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(
    input_ids, 
    max_length=256, 
    num_return_sequences=1, 
    pad_token_id=tokenizer.eos_token_id,
    no_repeat_ngram_size=2,
    early_stopping=True,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
answer = generated_text.split("Answer:")[1].strip() if "Answer:" in generated_text else generated_text
print(answer)

/Users/praneshjayasundar/Documents/Gunner/Boston-University/Fall-2023/pranesh-venv-nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


You should take: Cetirizine, Cefuroxime, Zyrtec, and Levocetrizine. These medications are prescribed to treat cough, cold and flu, as well as to prevent and treat any other symptoms.


In [7]:
question = "I have mouth ulcer, what should I do??"
input_text = f"Question: {question} Answer:"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(
    input_ids, 
    max_length=256, 
    num_return_sequences=1, 
    pad_token_id=tokenizer.eos_token_id,
    no_repeat_ngram_size=2,
    early_stopping=True,
    do_sample=True,
    temperature=0.8,
    top_k=50,
    top_p=0.95
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
answer = generated_text.split("Answer:")[1].strip() if "Answer:" in generated_text else generated_text
print(answer)

Based on your symptoms, I would like to do a few medical tests. First, you will need to take a complete physical skin exam and then we will start with a radiographic imaging procedure. Then, we may also need a plain x-ray to evaluate any damage to the ulcers.
